In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics, preprocessing

import random
from collections import defaultdict
from tqdm.notebook import tqdm
import copy


import torch
from torch import nn, optim, Tensor
from torch_sparse import SparseTensor, matmul

from torch_geometric.nn.conv.gcn_conv import gcn_norm 
from torch_geometric.nn.conv import MessagePassing


## Graph construction

Our graph should be represented as an adjacency matrix and at the same times capture the interaction between $u_i$ and $i_j$. Initially we have an user-item interaction matrix $R$ that does capture the $u_i$ and $i_j$ interaction. Later we need to construct an adjacency matrix $A$ from $R$. We can achieve it by the following

$$A = \begin{pmatrix}0 & R \\ R^T & 0\end{pmatrix}$$

In [69]:
def df_to_edge_list(data, src_index_col, dst_index_col, link_index_col, rating_threshold=1):
    edge_index = None
    src = [user_id for user_id in data[src_index_col]]
    
    dst = [item_id for item_id in data[dst_index_col]]
    link_vals = data[link_index_col].values
    
    edge_attr = torch.from_numpy(data[link_index_col].values).view(-1, 1).to(torch.long) >= rating_threshold
    
    edge_values = []
    
    edge_index = [[], []]
    for i in range(edge_attr.shape[0]):
        if edge_attr[i]:
            edge_index[0].append(src[i])
            edge_index[1].append(dst[i])
            edge_values.append(link_vals[i])
    
    return edge_index, edge_values

In [70]:
def conv_R_to_A(input_edge_index, input_edge_value, num_users, num_items):
    R = torch.zeros((num_users, num_items))
    for i in range(len(input_edge_index[0])):
        row_idx = input_edge_index[0][i]
        col_idx = input_edge_index[1][i]
        R[row_idx][col_idx] = input_edge_value[i]
    
    R_T = torch.transpose(R, 0, 1)
    
    A = torch.zeros((num_users + num_items, num_users + num_items))
    A[:num_users, num_users:] = R.clone()
    A[num_users:, :num_users] = R_T.clone()
    
    A_coo = A.to_sparse_coo()
    A_coo_indices = A_coo.indices()
    A_coo_values = A_coo.values()
    
    return A_coo_indices, A_coo_values

In [113]:
def conv_A_to_R(input_edge_index, input_edge_value, num_users, num_items):
    # print(input_edge_value.size(0), input_edge_index[0].size(0))
    assert input_edge_value.size(0) == input_edge_index[0].size(0)
    
    sparse_input_edge_index = SparseTensor(row=input_edge_index[0], 
                                           col=input_edge_index[1], 
                                           value=input_edge_value, 
                                           sparse_sizes=((num_users + num_items, num_users + num_items)))
    
    A = sparse_input_edge_index.to_dense()
    R = A[:num_users, num_users:]
    
    R_coo = R.to_sparse_coo()
    R_coo_indices = R_coo.indices()
    R_coo_values = R_coo.values()

    return R_coo_indices, R_coo_values

## Preprocess

In [2]:
DATA_PATH = "../data/interim/data.csv"

In [3]:
data = pd.read_csv(DATA_PATH)
data

,user,item,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [4]:
user_label_encoder = preprocessing.LabelEncoder()
movie_label_encoder = preprocessing.LabelEncoder()

data.user = user_label_encoder.fit_transform(data.user.values)
data.item = user_label_encoder.fit_transform(data.item.values)

In [5]:
data.user.max(), data.item.max()

(942, 1681)

Convert the data table to COO graph representation. In COO an edge $i$ is represented as a tuple $(\text{src}_i, \text{dst}_i, \text{value}_i)$. In this case we will represent them as $(\text{src}_i, \text{dst}_i)$ and $(\text{value}_i)$

In [9]:
edge_index, edge_value = df_to_edge_list(data, 'user', 'item', 'rating') 

In [10]:
print(edge_index)
print(edge_value)

[[195, 185, 21, 243, 165, 297, 114, 252, 304, 5, 61, 285, 199, 209, 223, 302, 121, 193, 290, 233, 118, 166, 298, 290, 307, 94, 37, 101, 62, 159, 49, 300, 224, 289, 96, 156, 180, 277, 275, 6, 9, 283, 200, 275, 286, 245, 241, 248, 98, 177, 250, 80, 259, 24, 58, 71, 86, 289, 41, 291, 114, 19, 200, 12, 245, 137, 166, 59, 56, 222, 188, 242, 91, 245, 193, 240, 177, 253, 292, 126, 224, 298, 224, 275, 290, 221, 266, 41, 10, 94, 7, 161, 86, 278, 144, 118, 61, 61, 27, 134, 31, 89, 285, 292, 215, 165, 249, 270, 159, 264, 197, 41, 167, 109, 57, 89, 270, 61, 278, 236, 93, 127, 297, 43, 263, 193, 71, 221, 249, 40, 223, 81, 261, 292, 215, 249, 58, 285, 243, 6, 173, 86, 193, 81, 12, 12, 243, 304, 94, 42, 298, 56, 83, 268, 298, 193, 159, 98, 9, 258, 84, 302, 212, 120, 89, 48, 41, 154, 67, 171, 18, 267, 4, 304, 43, 42, 278, 79, 253, 297, 278, 65, 17, 267, 98, 12, 25, 6, 221, 199, 118, 212, 275, 93, 129, 37, 159, 292, 25, 129, 91, 255, 0, 71, 55, 12, 14, 91, 206, 291, 231, 250, 223, 180, 258, 304, 51, 16

Convert to tensor

In [11]:
edge_index = torch.LongTensor(edge_index)
edge_value = torch.tensor(edge_value)

In [12]:
print(edge_index.size())
print(edge_value.size())

torch.Size([2, 100000])
torch.Size([100000])


In [15]:
num_users = len(data['user'].unique())
num_items = len(data['item'].unique())
print("num user:", num_users, "num items:", num_items)

num user: 943 num items: 1682


### Train/Test/Val split (0.8/0.1/0.1)

In [87]:
num_interactions = edge_index.shape[1]
print("Number of edges:", num_interactions)

Number of edges: 100000


In [88]:
all_indices = [i for i in range(num_interactions)]

train_indices, test_indices = train_test_split(all_indices, test_size=0.2, random_state=1337)
val_indices, test_indices = train_test_split(test_indices, test_size=0.5, random_state=1337)

In [89]:
train_edge_index = edge_index[:, train_indices]
train_edge_value = edge_value[train_indices]

val_edge_index = edge_index[:, val_indices]
val_edge_value = edge_value[val_indices]

test_edge_index = edge_index[:, test_indices]
test_edge_value = edge_value[test_indices]

In [90]:
train_edge_index, train_edge_value = conv_R_to_A(train_edge_index, train_edge_value, num_users=num_users, num_items=num_items)
val_edge_index, val_edge_value = conv_R_to_A(val_edge_index, val_edge_value, num_users=num_users, num_items=num_items)
test_edge_index, test_edge_value = conv_R_to_A(test_edge_index, test_edge_value, num_users=num_users, num_items=num_items)

In [111]:
print("train_edge_index shape", train_edge_index.size(), "train_edge_value shape", train_edge_value.size())
print("val_edge_index shape", train_edge_index.size(), "val_edge_value shape", train_edge_value.size())

train_edge_index shape torch.Size([2, 160000]) train_edge_value shape torch.Size([160000])
val_edge_index shape torch.Size([2, 160000]) val_edge_value shape torch.Size([160000])


## Model

- Architecture: LightGCN
- Loss: RMSE
- Optimizer: 
- Training steps:

In [116]:
class LightGCN(MessagePassing):
    def __init__(self, num_users, num_items, embedding_dim=64, K=3, add_self_loops=False, dropout_rate=0.1):
        super().__init__()
        self.dropout_rate = dropout_rate
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.K = K
        self.add_self_loops = add_self_loops
        
        # user and item embeddings for direct look up
        self.users_emb = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.embedding_dim)
        self.items_emb = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.embedding_dim)
        
        # initialize the input tensors with normally distributed values 
        nn.init.normal_(self.users_emb.weight, std=0.1)
        nn.init.normal_(self.items_emb.weight, std=0.1)
        
        # output predicted rating
        self.out = nn.Linear(self.embedding_dim + self.embedding_dim, 1)
    
    def forward(self, edge_index, edge_value):
        edge_index_norm = gcn_norm(edge_index=edge_index, add_self_loops=self.add_self_loops)
        
        emb_0 = torch.cat([self.users_emb.weight, self.items_emb.weight]) # E^0
        
        embs = [emb_0]
        
        # this emb will be pushed through graph layer
        emb_k = emb_0 
        
        # 
        for i in range(self.K):
            emb_k = self.propagate(edge_index=edge_index_norm[0], x=emb_k, norm=edge_index_norm[1])
            embs.append(emb_k)
            
        embs = torch.stack(embs, dim=1)
        
        emb_final = torch.mean(embs, dim=1)
        
        users_emb_final, items_emb_final = torch.split(emb_final, [self.num_users, self.num_items])
        
        # convert A to R (edge_index, edge_values)
        r_mat_edge_index, _ = conv_A_to_R(edge_index, edge_value, self.num_users, self.num_items) 
        
        src, dest = r_mat_edge_index[0], r_mat_edge_index[1]
        
        user_embeds = users_emb_final[src]
        item_embeds = items_emb_final[dest]
        
        output = torch.cat([user_embeds, item_embeds], dim=1)
        
        output = self.out(output)
        
        return output
        

In [133]:
def get_recall_at_k(input_edge_index, input_edge_values, pred_rating, k=10, threshold=3.5):
    with torch.no_grad():
        user_item_rating_list = defaultdict(list)
        
        for i in range(len(input_edge_index[0])):
            src = input_edge_index[0][i].item()
            dest = input_edge_index[1][i].item()
            true_rating = input_edge_values[i].item()
            pred_rating = pred_rating[i].item()
            
            user_item_rating_list[src].append((pred_rating, true_rating))
            
        recalls = dict()
        precisions = dict()
        
        for user_id, users_ratings in user_item_rating_list.items():
            users_ratings.sort(key=lambda x: x[0], reverse=True)
            
            n_rel = sum((true_r >= threshold) for (_, true_r) in users_ratings)
            n_rec_k = sum((est >= threshold) for (est, _) in users_ratings[:k])
            
            n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in users_ratings[:k])
            
            precisions[user_id] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
            recalls[user_id] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
        
        total_recall = sum(rec for rec in recalls.values()) / len(recalls)
        total_precission = sum(prec for prec in precisions.values()) / len(precisions)
        
        return total_recall, total_precission

In [134]:
layers = 1
model = LightGCN(num_users=num_users, num_items=num_items, K=layers)

In [137]:
ITERATIONS = 1_000
EPOCHS = 10

BATCH_SIZE = 1024
LR = 1e-3
ITERS_PER_EVAL = 200
ITERS_PER_LR_DECAY = 200
K = 10
LAMBDA = 1e-6

In [138]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
device

device(type='cpu')

In [139]:
model = model.to(device)
model.train()

optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay = 0.01)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

edge_index = edge_index.to(device)
train_edge_index = train_edge_index.to(device)
train_edge_value = train_edge_value.to(device)
val_edge_index = val_edge_index.to(device)
val_edge_value = val_edge_value.to(device)

loss_fn = nn.MSELoss()

In [140]:
# A matrices to R
R_train_edge_index, R_train_edge_value = conv_A_to_R(train_edge_index, train_edge_value, num_users=num_users, num_items=num_items)
R_val_edge_index, R_val_edge_value = conv_A_to_R(val_edge_index, val_edge_value, num_users=num_users, num_items=num_items)
# R_test_edge_index, R_test_edge_value = conv_R_to_A(test_edge_index, test_edge_value, num_users=num_users, num_items=num_items)

In [141]:
train_losses = []
val_losses = []
val_racall_at_ks = []

for iter in tqdm(range(ITERATIONS)):
    pred_ratings = model.forward(train_edge_index, train_edge_value)
    
    train_loss = loss_fn(pred_ratings, R_train_edge_value) # train R of values .view(-1, 1)
    
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
    
    if iter % ITERS_PER_EVAL == 0:
        model.eval()
        
        with torch.no_grad():
            val_pred_ratings = model.forward(val_edge_index, val_edge_value)
            
            val_losses = loss_fn(val_pred_ratings, R_val_edge_value.view(-1, 1))  # val R of values .view(-1, 1)
            recall_at_k, precision_at_k = get_recall_at_k(R_val_edge_index, R_val_edge_value, val_pred_ratings, k=20)
            
            val_racall_at_ks.append(round(recall_at_k, 5))
            train_losses.append(train_loss.item())
            val_losses.append(val_losses.item())
            
            print(f"Iteration {iter}/{ITERATIONS} | train loss: {round(train_loss, 5)} val loss: {round(train_loss, 5)} recall: {round(recall_at_k, 5)} precision {round(precision_at_k, 5)}")
        model.train()
    
    if iter % ITERS_PER_LR_DECAY == 0 and iter != 0:
        scheduler.step()

  0%|          | 0/1000 [00:00<?, ?it/s]

## Plot